In [ ]:
import custom_utils as cu
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt

import cv2

labels = ["green", "red", "yellow"]

In [ ]:
# prepare environment
import os
from dotenv import load_dotenv
# Load .env files
load_dotenv()

# Get environment variables
dataset_dir = os.getenv('DATASET_DIR')
ori_data_dir = os.getenv('TRAFFIC_LIGHT_ORIGINAL_DATA')
custom_data_dir = os.getenv('TRAFFIC_LIGHT_CUSTOM_DATA')

if not os.path.exists(ori_data_dir):
    print("Error: No original data set")

if not os.path.exists(custom_data_dir):
    os.mkdir(custom_data_dir)

for l in labels:
    label_dir = os.path.join(custom_data_dir, l)
    if not os.path.exists(label_dir):
        os.mkdir(label_dir)

In [ ]:
# step 0: get images 
img_dic = {}
for l in labels: 
    paths, images = cu.get_images(os.path.join(ori_data_dir, l), 2)
    print(paths)
    print(os.path.join(ori_data_dir, l))
    img_dic.update({l: images })

In [ ]:
def extract_boxes(yolo_res, cls):
    ex_boxes = []
    for res in yolo_res:
        for i, c in enumerate(res.boxes.cls.numpy()):
            if c == cls:
                points = res.boxes.xyxyn.numpy()[i]
                ex_box = cu.extract_rectangle_from_image(res.orig_img, points) 
                ex_boxes.append(ex_box)
    return ex_boxes

In [ ]:
def average_resize(res_dict):
    #dia_len = [(i.shape[1]**2 + i.shape[2])**0.5 for i in images]
    hights = []
    widths = []
    for l in res_dict:
        hights.append([i.shape[0] for i in res_dict[l]])
        widths.append([i.shape[1] for i in res_dict[l]])
    #height, width = image.shape[:2]
    avr_x = int(np.average(widths))
    avr_y = int(np.average(hights))
    for l in res_dict:
        for idx, i in enumerate(res_dict[l]):
            res_dict[l][idx] = cv2.resize(i, (avr_x,avr_y))
        #cv2.imwrite(os.path.join(custom_data_dir, label, str(idx), ".jpg"), i) # not nessecary that label is in the name of image
    return res_dict 

In [ ]:
model = YOLO("yolov8m.pt") # test with sample images shows that, m is a good mid way between accuracy and run time
res_dict = {}

In [ ]:
# step 1: analyze images and extract boxes with yolo
for l in labels:
    res = model.predict(img_dic[l], conf=0.3)
    boxes = extract_boxes(res,9)
    res_dict.update({l : boxes})

In [ ]:
# resize all images to the same average hight and width 
res_dict = average_resize(res_dict) 

In [ ]:
# save resized images in custom_data

In [ ]:
#print(type(res_dict["red"]))
#i = res_dict["yellow"][0]
#plt.imshow(i)

In [ ]:
# todo 
# drop similar boxes is requiered becaus some times yolo identify the same object with a different bounding box
"""
def drop_similar_boxes(input_list, threshold):
    def are_elements_similar(elem1, elem2):
        x1_diff = abs(elem1[0] - elem2[0])
        y1_diff = abs(elem1[1] - elem2[1])
        x2_diff = abs(elem1[2] - elem2[2])
        y2_diff = abs(elem1[3] - elem2[3])
        return x1_diff < threshold or y1_diff < threshold or x2_diff < threshold or y2_diff < threshold

    unique_elements = []
    for elem in input_list:
        is_similar = False
        for unique_elem in unique_elements:
            if are_elements_similar(elem, unique_elem):
                is_similar = True
                break
        if not is_similar:
            unique_elements.append(elem)
    return unique_elements
"""
"""
input_list = [[1.0, 2.0, 3.0, 4.0], [1.2, 2.2, 3.0, 4.0], [5.0, 6.0, 7.0, 8.0], [1.1, 2.1, 3.1, 4.1]]
threshold = 0.2
filtered_list = drop_similar_boxes(input_list, threshold)
print(filtered_list)
"""

In [ ]:
# step 2: extract all traffic lights from all analyzed images and create for each an new image

# Process results list
image_name = 'traffic_light_green'
file_type = '.jpg'

counter = 0
extracted_dic = {}
for l in labels:
    print(l)
    ex_images = [] # is an list because probably there are more traffic_lights objects in a picture
    for j, result in enumerate(res_dict[l]):
        for i,c in enumerate(result.boxes.cls.numpy()):
            # find traffic light boxes

            if c == 9:
                points = result.boxes.xyxyn.numpy()[i]
                extracted_image = cu.extract_rectangle_from_image(images[j], points) 
                ex_images.append(extracted_image)
                print(extracted_image)
                # res_dict[l][j] = extracted #
                # print(type(res_dict[l][j]))
                counter += 1
    
    extracted_dic.update({l : ex_images})

"""
# resize imgaes
print(extracted_dic[l])
average_resize(l, extracted_dic[l])
out_path = os.path.join(custom_data_dir, l)
print(out_path)
cu.write_images(extracted_dic[l], out_path, main_name=l)
"""

# step 3: check images manually and label them

In [ ]:
cv2.imread()

In [ ]:
for i in res_dict:
    print(i)

In [ ]:
import os
from dotenv import load_dotenv
import matplotlib.pyplot as plt
load_dotenv()
p = os.getenv('TRAFFIC_LIGHT_ORIGINAL_DATA')
p = os.path.join(p, "green")
paths, imgs = cu.get_images(p,3)
i = cv2.cvtColor(imgs[0], cv2.COLOR_BGR2RGB)
plt.imshow(i)